# Review data preparation

In [1]:
%load_ext autoreload
%autoreload 2

# Import ds libraries
import pandas as pd
import numpy as np
import re

from datetime import datetime, timedelta

# Import acquire functions
import nick_acquire as a
import nick_prepare as prep

In [219]:
#api_reviews = pd.read_csv('reviews_progress.csv')
scrape_reviews = pd.read_csv('scraped_reviews.csv')

In [3]:
scrape_reviews.head(3)

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional
0,50138517,ChZDSUhNMG9nS0VJQ0FnSUNGcGF5bFV3EAE,NaN,5 days ago,2023-11-04 04:31:32.813385,5.0,Christopher Croft,40,https://www.google.com/maps/contrib/1158098693...,"['Service: 5', 'Atmosphere: 5']"
1,50138517,ChdDSUhNMG9nS0VJQ0FnSUQ1MGVhUHRnRRAB,"Broth salty, cloudy. Summer rolls old. Short o...",a week ago,2023-11-04 04:31:32.814883,2.0,D HH,1,https://www.google.com/maps/contrib/1047236894...,"['Service Dine in', 'Meal type Lunch', 'Price ..."
2,50138517,ChdDSUhNMG9nS0VJQ0FnSUQ1X0xQamdBRRAB,"Had the brisket and flank pho, the broth and n...",2 weeks ago,2023-11-04 04:31:32.815647,5.0,Eling Lee,29,https://www.google.com/maps/contrib/1180366842...,"['Service Dine in', 'Meal type Lunch', 'Price ..."


In [53]:
def clean_additional(data):
    df = data.copy()
    df['additional'] = df.r_additional.apply(lambda x: re.findall(r'\'([^\']+)?\'', x))
    important_additionals = \
    ['Service: 5', 'Service: 4', 'Service: 3','Service: 2','Service: 1', 
     'Atmosphere: 5', 'Atmosphere: 4', 'Atmosphere: 3', 'Atmosphere: 2', 'Atmosphere: 1', 
     'Food: 5',  'Food: 4',  'Food: 3',  'Food: 2',  'Food: 1', 'Price per person $10–20',
     'Price per person $20–30', 'Price per person $30–50','Price per person $50–100', 'Price per person $100+']
    df['additional'] = df.additional.apply(lambda x: [ele for ele in x if ele in important_additionals])
    df['additional'] = df.additional.apply(lambda x: ' '.join(x))
    return df

In [194]:
clean = clean_additional(scrape_reviews)
clean.head(3)

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional,additional
0,50138517,ChZDSUhNMG9nS0VJQ0FnSUNGcGF5bFV3EAE,NaN,5 days ago,2023-11-04 04:31:32.813385,5.0,Christopher Croft,40,https://www.google.com/maps/contrib/1158098693...,"['Service: 5', 'Atmosphere: 5']",Service: 5 Atmosphere: 5
1,50138517,ChdDSUhNMG9nS0VJQ0FnSUQ1MGVhUHRnRRAB,"Broth salty, cloudy. Summer rolls old. Short o...",a week ago,2023-11-04 04:31:32.814883,2.0,D HH,1,https://www.google.com/maps/contrib/1047236894...,"['Service Dine in', 'Meal type Lunch', 'Price ...",Price per person $20–30 Food: 2 Service: 4 Atm...
2,50138517,ChdDSUhNMG9nS0VJQ0FnSUQ1X0xQamdBRRAB,"Had the brisket and flank pho, the broth and n...",2 weeks ago,2023-11-04 04:31:32.815647,5.0,Eling Lee,29,https://www.google.com/maps/contrib/1180366842...,"['Service Dine in', 'Meal type Lunch', 'Price ...",Price per person $10–20 Food: 5 Service: 5 Atm...


In [177]:
def create_additional_columns(df):
    clean = df.copy()
    clean['service'] = \
    [int(re.findall(r'Service: (\d)', x)[0]) if re.search(r'Service: (\d)', x) else None for x in clean.additional]

    clean['atmosphere'] = \
    [int(re.findall(r'Atmosphere: (\d)', x)[0]) if re.search(r'Atmosphere: (\d)', x) else None for x in clean.additional]

    clean['food'] = \
    [int(re.findall(r'Food: (\d)', x)[0]) if re.search(r'Food: (\d)', x) else None for x in clean.additional]

    clean['price_per_person'] = \
    [re.findall(r'\$([^\s]+)', x)[0] if re.search(r'\$([^\s]+)', x) else None for x in clean.additional]
    
    return clean

In [195]:
clean = create_additional_columns(clean)

In [187]:
clean.head()

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional,additional,service,atmosphere,food,price_per_person
0,50138517,ChZDSUhNMG9nS0VJQ0FnSUNGcGF5bFV3EAE,NaN,5 days ago,2023-11-04 04:31:32.813385,5.0,Christopher Croft,40,https://www.google.com/maps/contrib/1158098693...,"['Service: 5', 'Atmosphere: 5']",Service: 5 Atmosphere: 5,5.0,5.0,NaN,None
1,50138517,ChdDSUhNMG9nS0VJQ0FnSUQ1MGVhUHRnRRAB,"Broth salty, cloudy. Summer rolls old. Short o...",a week ago,2023-11-04 04:31:32.814883,2.0,D HH,1,https://www.google.com/maps/contrib/1047236894...,"['Service Dine in', 'Meal type Lunch', 'Price ...",Price per person $20–30 Food: 2 Service: 4 Atm...,4.0,3.0,2.0,20–30
2,50138517,ChdDSUhNMG9nS0VJQ0FnSUQ1X0xQamdBRRAB,"Had the brisket and flank pho, the broth and n...",2 weeks ago,2023-11-04 04:31:32.815647,5.0,Eling Lee,29,https://www.google.com/maps/contrib/1180366842...,"['Service Dine in', 'Meal type Lunch', 'Price ...",Price per person $10–20 Food: 5 Service: 5 Atm...,5.0,5.0,5.0,10–20
3,50138517,ChdDSUhNMG9nS0VJQ0FnSUM1ak1pTTd3RRAB,NaN,3 weeks ago,2023-11-04 04:31:32.816426,5.0,Robert Longtin,90,https://www.google.com/maps/contrib/1152111335...,"['Meal type Lunch', 'Price per person $10–20',...",Price per person $10–20 Food: 5 Service: 5 Atm...,5.0,5.0,5.0,10–20
4,50138517,ChZDSUhNMG9nS0VJQ0FnSUM1cEl1RllREAE,I was waiting in front of the counter for at l...,3 weeks ago,2023-11-04 04:31:32.817012,1.0,Jim L,0,https://www.google.com/maps/contrib/1024033632...,"['Service Take out', 'Meal type Lunch', 'Servi...",Service: 1,1.0,NaN,NaN,None


In [161]:
def check_cols(n):
    df = clean[clean.additional != ''].reset_index()
    print(df.additional[n])
    print()
    print(df[df.additional != ''][['service', 'atmosphere', 'food', 'price_per_person']].iloc[n])

In [164]:
check_cols(743)

Price per person $10–20

service              None
atmosphere           None
food                 None
price_per_person    10–20
Name: 743, dtype: object


In [208]:
def adjust_prices(data):
    df = data.copy()
    new_prices = []

    for price in df.price_per_person:
        if price == '10–20':
            #print(price)
            new_prices.append(15)
        elif price == '20–30':
            #print(price)
            new_prices.append(25)
        elif price == '30–50':
            new_prices.append(40)
        elif price == '50–100':
            new_prices.append(75)
        elif price == '100+':
            new_prices.append(100)
        else:
            new_prices.append(None)
    df.price_per_person = new_prices
    return df

In [210]:
clean = adjust_prices(clean)

In [190]:
clean

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional,additional,service,atmosphere,food,price_per_person
0,50138517,ChZDSUhNMG9nS0VJQ0FnSUNGcGF5bFV3EAE,NaN,5 days ago,2023-11-04 04:31:32.813385,5.0,Christopher Croft,40,https://www.google.com/maps/contrib/1158098693...,"['Service: 5', 'Atmosphere: 5']",Service: 5 Atmosphere: 5,5.0,5.0,NaN,NaN
1,50138517,ChdDSUhNMG9nS0VJQ0FnSUQ1MGVhUHRnRRAB,"Broth salty, cloudy. Summer rolls old. Short o...",a week ago,2023-11-04 04:31:32.814883,2.0,D HH,1,https://www.google.com/maps/contrib/1047236894...,"['Service Dine in', 'Meal type Lunch', 'Price ...",Price per person $20–30 Food: 2 Service: 4 Atm...,4.0,3.0,2.0,NaN
2,50138517,ChdDSUhNMG9nS0VJQ0FnSUQ1X0xQamdBRRAB,"Had the brisket and flank pho, the broth and n...",2 weeks ago,2023-11-04 04:31:32.815647,5.0,Eling Lee,29,https://www.google.com/maps/contrib/1180366842...,"['Service Dine in', 'Meal type Lunch', 'Price ...",Price per person $10–20 Food: 5 Service: 5 Atm...,5.0,5.0,5.0,NaN
3,50138517,ChdDSUhNMG9nS0VJQ0FnSUM1ak1pTTd3RRAB,NaN,3 weeks ago,2023-11-04 04:31:32.816426,5.0,Robert Longtin,90,https://www.google.com/maps/contrib/1152111335...,"['Meal type Lunch', 'Price per person $10–20',...",Price per person $10–20 Food: 5 Service: 5 Atm...,5.0,5.0,5.0,NaN
4,50138517,ChZDSUhNMG9nS0VJQ0FnSUM1cEl1RllREAE,I was waiting in front of the counter for at l...,3 weeks ago,2023-11-04 04:31:32.817012,1.0,Jim L,0,https://www.google.com/maps/contrib/1024033632...,"['Service Take out', 'Meal type Lunch', 'Servi...",Service: 1,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325198,50075147,ChdDSUhNMG9nS0VJQ0FnSUNZai1mc3pBRRAB,Fun place to visit,4 years ago,2023-11-04 03:05:55.140870,5.0,David Kosnitzky,47,https://www.google.com/maps/contrib/1176611176...,[],,NaN,NaN,NaN,NaN
325199,50075147,ChdDSUhNMG9nS0VJQ0FnSUNZaTd2ZnJnRRAB,NaN,4 years ago,2023-11-04 03:05:55.141305,5.0,Kaila Pimentel,0,https://www.google.com/maps/contrib/1183717723...,[],,NaN,NaN,NaN,NaN
325200,50075147,ChZDSUhNMG9nS0VJQ0FnSUNZa19LQkJBEAE,"Excellent food, small cozy restaurant",4 years ago,2023-11-04 03:05:55.141783,5.0,Scott Allen,252,https://www.google.com/maps/contrib/1038751070...,[],,NaN,NaN,NaN,NaN
325201,50075147,ChRDSUhNMG9nS0VJQ0FnSUNZaUwwdRAB,"Very good food, try it and you will see.",4 years ago,2023-11-04 03:05:55.142285,5.0,Gustavo Infante,0,https://www.google.com/maps/contrib/1138579259...,[],,NaN,NaN,NaN,NaN


In [139]:
clean[['service', 'atmosphere', 'food', 'price_per_person']]

,service,atmosphere,food,price_per_person
0,5,5,None,None
1,4,3,2,20–30
2,5,5,5,10–20
3,5,5,5,10–20
4,1,None,None,None
...,...,...,...,...
325198,None,None,None,None
325199,None,None,None,None
325200,None,None,None,None
325201,None,None,None,None


In [77]:
re.match(r'Service: (\d)', clean.additional[o])

In [52]:
additional = scrape_reviews.r_additional.apply(lambda x: re.findall(r'\'([^\']+)?\'', x))
a = [j for i in additional for j in i]
pd.Series(a).value_counts()[:5]

Service: 5          39583
Atmosphere: 5       35733
Service Dine in     35316
Food: 5             32461
Meal type Dinner    31864
Name: count, dtype: int64

In [48]:
important_additionals = ['Service: 5', 'Service: 4', 'Service: 3','Service: 2','Service: 1', 
                         'Atmosphere: 5', 'Atmosphere: 4', 'Atmosphere: 3', 'Atmosphere: 2', 'Atmosphere: 1', 
                         'Food: 5',  'Food: 4',  'Food: 3',  'Food: 2',  'Food: 1', 'Price per person $10–20',
                         'Price per person $20–30', 'Price per person $30–50','Price per person $50–100', 'Price per person $100+']

In [39]:
clean_additional = scrape_reviews.additional.apply(lambda x: [ele for ele in x if ele in important_additionals])

In [41]:
def clean_api_reviews(api_data):
    df = api_data.copy()
    cols = ['camis', 'publish_time', 'review_text', 'review_rating']
    df = df[cols]
    return df


In [ ]:
def clean_dates(data):
    scrape_reviews = data.copy()
    scrape_reviews.relative_date = scrape_reviews.relative_date.apply(lambda x: x[:-4])
    scrape_reviews.relative_date = ['1 years' if date == 'a year' else date for date in scrape_reviews.relative_date]
    scrape_reviews.relative_date = [re.sub(r'^a', '1', date) if date[0] == 'a' else date for date in scrape_reviews.relative_date]
    return scrape_reviews

In [ ]:
reviews = clean_dates(scrape_reviews)

In [ ]:
def adjust_dates(scrape_reviews):
    
    dataframes = []  # Create empty list to store dataframes
    
    # Isolate each restaurant by id
    for restaurant_id in scrape_reviews.id.unique():
        # Create dataframe of ONE restaurant
        restaurant = scrape_reviews[scrape_reviews.id == restaurant_id].copy()
        
        # Create df of review counts per relative_date and calculate average distribution of reviews
        place = scrape_reviews[scrape_reviews.id == restaurant_id]
        review_counts = pd.DataFrame(place.relative_date.value_counts())
        review_counts['increment'] = 365/review_counts.relative_date
        
        # Create empty list for new dates, i variable to count increments, and previous_year to track year 
        new_dates = []
        i = 0
        previous_year = '1 years'

        for date in restaurant.relative_date: 
            if 'years' in date:  # If date is in years, function will adjust it to estimated date
                if date != previous_year:  # When date changes from 'x years' to 'x + 1 years' counters are reset 
                    i = 0
                    previous_year = date
                # Calculate adjusted date
                adjusted_date = (365*(int(re.findall(r'\d+', date)[0]))) + (review_counts.loc[date].increment * i)
                i += 1
                new_dates.append(str(round(adjusted_date)))  # Append adjsuted date
            else:
                new_dates.append(date)  # Append normal date if date < 1 year
        restaurant['new_date'] = new_dates  # Replace dates with new_dates
        dataframes.append(restaurant)  # Append dataframe to list of dataframes
    reviews = pd.concat(dataframes)  # Join all dataframes
    return reviews  # Return joined data


In [ ]:
s = pd.read_csv('scraped_reviews.csv')

In [ ]:
place = s[s.id == 41375676]
review_counts = pd.DataFrame(place.relative_date.value_counts())
#review_counts.rename(columns={review_counts.columns[0]:'new_name'})
# review_counts['increment'] = 365 / review_counts.index
review_counts

In [ ]:
reviews = prep.adjust_dates(reviews)

In [ ]:
def calculate_days(data):
    reviews = data.copy()
    new_date = []
    for date in reviews.new_date:
        #print(date)
        unit = re.sub(r'[^a-z]', '', date)
        if 'hour' in unit:
            new_date.append('1')
        elif 'day' in unit:
            new_date.append(re.sub(r'[^0-9]', '', date))
        elif 'week' in unit:
            new_date.append(int(re.sub(r'[^0-9]', '', date))*7)
        elif 'month' in unit:
            new_date.append(int(re.sub(r'[^0-9]', '', date))*30)
        else:
            new_date.append(date)

    reviews['newer_dates'] = new_date
    reviews['final_date'] = [pd.to_datetime(retrieval_date) - timedelta(days = n) for retrieval_date,n in zip(reviews.retrieval_date, reviews.newer_dates.astype(int))]
    return reviews

In [ ]:
final_df = calculate_days(reviews)

In [213]:
def clean_reviews(data):
    final_df = data.copy()
    cols = ['id', 'final_date', 'caption', 'rating','service', 'atmosphere', 'food',
                        'price_per_person']
    final_df = final_df[cols]
    final_df.rating = final_df.rating.astype(int)
    final_df.columns = ['camis', 'publish_time', 'review_text', 'review_rating', 'service', 'atmosphere', 'food',
                        'price_per_person']
    return final_df

In [ ]:
final_df = clean_reviews(final_df)

In [ ]:
final_df.head()

In [222]:
#api_reviews = pd.read_csv('reviews_progress.csv')
scrape_reviews = pd.read_csv('scraped_reviews.csv')

In [226]:
scrape_reviews

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional
0,50138517,ChZDSUhNMG9nS0VJQ0FnSUNGcGF5bFV3EAE,NaN,5 days ago,2023-11-04 04:31:32.813385,5.0,Christopher Croft,40,https://www.google.com/maps/contrib/1158098693...,"['Service: 5', 'Atmosphere: 5']"
1,50138517,ChdDSUhNMG9nS0VJQ0FnSUQ1MGVhUHRnRRAB,"Broth salty, cloudy. Summer rolls old. Short o...",a week ago,2023-11-04 04:31:32.814883,2.0,D HH,1,https://www.google.com/maps/contrib/1047236894...,"['Service Dine in', 'Meal type Lunch', 'Price ..."
2,50138517,ChdDSUhNMG9nS0VJQ0FnSUQ1X0xQamdBRRAB,"Had the brisket and flank pho, the broth and n...",2 weeks ago,2023-11-04 04:31:32.815647,5.0,Eling Lee,29,https://www.google.com/maps/contrib/1180366842...,"['Service Dine in', 'Meal type Lunch', 'Price ..."
3,50138517,ChdDSUhNMG9nS0VJQ0FnSUM1ak1pTTd3RRAB,NaN,3 weeks ago,2023-11-04 04:31:32.816426,5.0,Robert Longtin,90,https://www.google.com/maps/contrib/1152111335...,"['Meal type Lunch', 'Price per person $10–20',..."
4,50138517,ChZDSUhNMG9nS0VJQ0FnSUM1cEl1RllREAE,I was waiting in front of the counter for at l...,3 weeks ago,2023-11-04 04:31:32.817012,1.0,Jim L,0,https://www.google.com/maps/contrib/1024033632...,"['Service Take out', 'Meal type Lunch', 'Servi..."
...,...,...,...,...,...,...,...,...,...,...
371100,50066463,ChdDSUhNMG9nS0VJQ0FnSUM0anAzRndBRRAB,NaN,4 years ago,2023-11-05 16:04:16.106500,4.0,Kesley,5,https://www.google.com/maps/contrib/1144807299...,[]
371101,50066463,ChZDSUhNMG9nS0VJQ0FnSUM0aXNEV1BREAE,Kinda pricey but great food n service,4 years ago,2023-11-05 16:04:16.107257,5.0,gixxercon39,0,https://www.google.com/maps/contrib/1111793398...,[]
371102,50066463,ChdDSUhNMG9nS0VJQ0FnSUM0bWJpWTlnRRAB,Perfect. Great food.,4 years ago,2023-11-05 16:04:16.108349,5.0,RICARDO DA SILVA DUARTE,102,https://www.google.com/maps/contrib/1092690370...,[]
371103,50066463,ChZDSUhNMG9nS0VJQ0FnSUM0akxpWEVREAE,I love it! There is a wide variety of food sol...,4 years ago,2023-11-05 16:04:16.109658,5.0,Cristina Abeledo,154,https://www.google.com/maps/contrib/1024912120...,[]


In [223]:
reviews = prep.cleanse_reviews(scrape_reviews)

In [229]:
prep.final_ny()

,camis,dba,boro,phone,inspection_date,latitude,longitude,cuisine_description,action,score,grade,full_address,violation_code,violation_description
0,50110919,TASHA'S ROTI SHOP,Queens,3474631035,2023-02-28T00:00:00.000,40.691470,-73.812484,Caribbean,No violations,0,A,13419 LIBERTY AVE 11419,No violation,No violation
1,50001488,MAR'S,Queens,7186852480,2020-01-31T00:00:00.000,40.758442,-73.924616,American,No violations,0,A,3421 34 AVE 11106,No violation,No violation
2,50042621,RAIN II,Manhattan,3475834044,2023-03-27T00:00:00.000,40.838255,-73.939889,Thai,Violations cited,7,A,1095 SAINT NICHOLAS AVENUE 10032,02B,Hot TCS food item not held at or above 140 °F.
3,50098128,TAKAHACHI,Manhattan,3479225061,2022-10-11T00:00:00.000,40.725166,-73.984191,Japanese,Violations cited,34,C,85 AVENUE A 10009,04L 06E 06C 06D 02B 06F,Evidence of mice or live mice in establishment...
4,50005848,IPPUDO,Manhattan,2123880088,2023-06-13T00:00:00.000,40.730947,-73.990435,Japanese,Violations cited,13,A,65 4 AVENUE 10003,02B 10F 10B,Hot TCS food item not held at or above 140 °F....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59984,50063071,ITAEWON POCHA,Manhattan,2127142266,2020-01-28T00:00:00.000,40.747723,-73.986877,Korean,Violations cited,5,A,28 WEST 32 STREET 10001,06E,"Sanitized equipment or utensil, including in-u..."
59985,40368577,DENNY'S PUB,Brooklyn,7184352156,2023-01-11T00:00:00.000,40.642932,-73.978870,American,Violations cited,8,A,106 BEVERLY ROAD 11218,04H,"Raw, cooked or prepared food is adulterated, c..."
59986,40934037,JIKJI CAFE,Queens,7187512003,2018-05-24T00:00:00.000,40.648313,-73.788281,Japanese,Violations cited,3,A,JFK INT'L AIRPORT 11430,10F,Non-food contact surface improperly constructe...
59987,50110252,BROTHER BAGELS,Brooklyn,7182224357,2021-11-19T00:00:00.000,40.684317,-73.994941,Bagels/Pretzels,Re-opened,5,A,285 COURT STREET 11231,10F,Non-food contact surface improperly constructe...


In [225]:
reviews

,camis,publish_time,review_text,review_rating,service,atmosphere,food,price_per_person
0,50138517,2023-10-30,,5,5.0,5.0,NaN,NaN
1,50138517,2023-10-28,"Broth salty, cloudy. Summer rolls old. Short o...",2,4.0,3.0,2.0,25.0
2,50138517,2023-10-21,"Had the brisket and flank pho, the broth and n...",5,5.0,5.0,5.0,15.0
3,50138517,2023-10-14,,5,5.0,5.0,5.0,15.0
4,50138517,2023-10-14,I was waiting in front of the counter for at l...,1,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
371100,50066463,2018-12-04,,4,NaN,NaN,NaN,NaN
371101,50066463,2018-11-28,Kinda pricey but great food n service,5,NaN,NaN,NaN,NaN
371102,50066463,2018-11-23,Perfect. Great food.,5,NaN,NaN,NaN,NaN
371103,50066463,2018-11-17,I love it! There is a wide variety of food sol...,5,NaN,NaN,NaN,NaN


In [227]:
reviews.to_csv('reviews.csv', index=False)

In [ ]:
data = pd.read_csv('reviews.csv')

In [ ]:

from pytz import timezone

In [ ]:
# reviews.publish_time contains more than one time format (zulu and naive)
# We must standardize the format

# Function to convert string to local time and format it
def convert_to_local_and_format(dt_str):
    # Define the local timezone (e.g., Eastern Time)
    local_tz = timezone('America/New_York')
    
    # Parse the string into a datetime object
    dt = pd.to_datetime(dt_str)

    # Check if datetime is timezone-aware (has valid timezone info)
    if dt.tzinfo is not None and dt.tzinfo.utcoffset(dt) is not None:
        # Convert to local timezone
        dt = dt.astimezone(local_tz)

    # Return only the date part
    return dt.date()

In [ ]:
# Apply the conversion and formatting to the 'publish_time' column.
data['publish_time'] = data['publish_time'].apply(convert_to_local_and_format) # The output is still a string

# Convert the 'publish_time' column to datetime, handling ISO8601 format
data['publish_time'] = pd.to_datetime(data['publish_time'])

In [ ]:
data.dtypes